<a href="https://colab.research.google.com/github/advik-7/Blogging-Automation/blob/main/Automatic_Blogging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Dependencies**


In [24]:
!pip install langgraph -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.0 MB/s eta 0:00:00


In [1]:
!pip install serpapi -qq

In [15]:
!pip install langchain_community -qq
!pip install pypdf -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install google-search-results -qq

  Preparing metadata (setup.py) ... done


In [3]:
!pip install google-serp-api -qq

In [16]:
!pip install --upgrade google-generativeai -qq

In [19]:
!pip install faiss-cpu -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.5 MB/s eta 0:00:00


In [27]:
!pip install langchain_google_genai -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00


In [28]:
import os
import google.generativeai as genai
from typing import Optional, List, Mapping, Any
from langchain_core.tools import tool
from langchain.llms.base import LLM
import os
from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph
from langchain.schema import AIMessage


In [5]:
from serpapi.google_search import GoogleSearch

#**Getting the Recent Trends**

In [6]:
def get_related_queries(keyword, api_key):
    """
    Fetch related queries for a given keyword using SerpApi Google Trends engine.

    Args:
        keyword (str): The keyword to search trends for.
        api_key (str): Your SerpApi API key.

    Returns:
        dict: A dictionary with 'rising' and 'top' related queries.
    """
    params = {
        "engine": "google_trends",
        "q": keyword,
        "data_type": "RELATED_QUERIES",
        "api_key": api_key,
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    if "related_queries" in results:
        related_queries = results["related_queries"]
        rising = [
            {"query": item["query"], "value": item["value"], "link": item["link"]}
            for item in related_queries.get("rising", [])
        ]
        top = [
            {"query": item["query"], "value": item["value"], "link": item["link"]}
            for item in related_queries.get("top", [])
        ]

        return {"rising": rising, "top": top}
    else:
        return {"error": "No related queries found or an issue occurred."}



In [7]:
 # Replace with your API key
keyword = "applied AI"
queries = get_related_queries(keyword, api_key)
print("Rising Queries:")
for query in queries["rising"]:
  print(f"- {query['query']} (Growth: {query['value']})")

  print("\nTop Queries:")
for query in queries["top"]:
    print(f"- {query['query']} (Interest: {query['value']})")

Rising Queries:
- ai humanizer (Growth: Breakout)

Top Queries:
- applied digital corp (Growth: Breakout)

Top Queries:
- nvidia stock price (Growth: Breakout)

Top Queries:
- outlier ai (Growth: Breakout)

Top Queries:
- which ai technology is commonly used to convert spoken language into text on devices like smartphones? (Growth: Breakout)

Top Queries:
- intel stock (Growth: Breakout)

Top Queries:
- big bear ai stock (Growth: Breakout)

Top Queries:
- palantir stock (Growth: Breakout)

Top Queries:
- serve robotics stock (Growth: Breakout)

Top Queries:
- recursion pharmaceuticals stock (Growth: Breakout)

Top Queries:
- sound hound ai stock (Growth: Breakout)

Top Queries:
- devin ai (Growth: Breakout)

Top Queries:
- gemini (Growth: +2,400%)

Top Queries:
- gemini ai (Growth: +1,950%)

Top Queries:
- applied digital (Growth: +80%)

Top Queries:
- what is ai (Interest: 100) [Link: https://trends.google.com/trends/explore?q=what+is+ai&date=today+12-m]
- what is applied ai (Interest

In [14]:
def get_breakout_queries(queries):
    # Extract breakout queries from both "rising" and "top" sections
    breakout_queries = []

    # Check the 'rising' section
    for query in queries.get("rising", []):
        if query.get("value") == "Breakout":
            breakout_queries.append(query)

    # Check the 'top' section
    for query in queries.get("top", []):
        if query.get("value") == "Breakout":
            breakout_queries.append(query)

    return breakout_queries

def choose_best_topic(queries):
    # Select the topic with the highest interest or growth
    best_topic = None
    best_value = -float('inf')

    for query in queries:
        value = query.get('value', '')
        # If it's a numeric growth value, try to convert it to a number
        if isinstance(value, str) and value != "Breakout":
            try:
                # Convert growth percentage string to a numeric value (remove % sign and commas)
                growth_value = float(value.replace('%', '').replace(',', ''))
                if growth_value > best_value:
                    best_value = growth_value
                    best_topic = query
            except ValueError:
                continue
        elif value == "Breakout" and best_topic is None:
            # If there's no topic yet, set the first Breakout as the best
            best_topic = query

    return best_topic

# Get breakout queries
breakout_queries = get_breakout_queries(queries)

# Choose the best topic (with the highest importance, based on growth or interest)
best_topic = choose_best_topic(breakout_queries)
print(best_topic['query'])


ai humanizer


#**Generating Basic Blog Structure**

In [22]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.llms.base import LLM
import requests
from sentence_transformers import SentenceTransformer
import numpy as np
import google.generativeai as genai
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Optional, List, Mapping, Any
import faiss


@tool
def retrieve_relevant_chunks(query, pdf_file_path, model_name, top_k=5):
    """Generalized function to retrieve relevant chunks using FAISS from a PDF document."""
    # Extract text as plain text from all pages
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()

    # Combine all document texts into a single string
    pdf_text = "\n".join([doc.page_content for doc in documents])

    # Split the text into chunks (you can fine-tune this logic based on your needs)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(pdf_text)

    # Assuming FAISS index is named after the PDF file (same directory as the PDF)
    faiss_index_path = pdf_file_path.replace('.pdf', '_faiss_index.bin')
    index = faiss.read_index(faiss_index_path)

    # Create the SentenceTransformer model to encode the query
    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query]).astype('float32')

    # Perform FAISS search to get the relevant chunks
    distances, indices = index.search(query_embedding, top_k)

    # Return the top-k relevant chunks
    return [chunks[i].strip() for i in indices[0]]

@tool
def query_llm(prompt: str, context: str = "") -> str:
    """LLM query tool for response generation using Gemini."""
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    model = genai.GenerativeModel("gemini-1.5-flash")
    complete_prompt = f"{context}\n\nQuestion: {prompt}\nAnswer:"
    response = model.generate_content([complete_prompt])
    return response.text
@tool
def generate_blog_structure_with_insights(query: str, pdf_file_path_1: str, pdf_file_path_2: str) -> str:
    """Generate the structure of the blog based on the query and insights from provided PDFs, focusing on the blog layout and sections."""

    # Retrieve insights on writing blogs from two different PDF sources
    insights_1 = retrieve_relevant_chunks(query, pdf_file_path_1, "sentence-transformers/LaBSE", top_k=3)
    insights_2 = retrieve_relevant_chunks(query, pdf_file_path_2, "all-MiniLM-L6-v2", top_k=3)

    # Combine insights from both PDFs
    combined_insights = "\n".join(insights_1 + insights_2)

    # Generate blog structure using these insights
    blog_structure_prompt = f"""
    You are a content strategist, and your task is to create a **structure** for a blog based on the following query. Your focus is **creating the framework**—not writing content—but organizing the blog into sections that are relevant and effective.

    Use the insights below to shape the **layout and structure** for the blog. The blog should be broken down into clear, actionable sections.

    **Insights:**
    {combined_insights}

    **Blog Topic:** {query}

    The blog should include the following sections:
    1. **A catchy title**
    2. **An introductory paragraph**—an overview of the blog topic
    3. **Subheadings**—key sections or topics that the blog will cover
    4. **Suggestions for images, logos, or media**—where to place visuals to enhance the blog
    5. **A conclusion**—what should the reader take away from the blog?

    Please provide only the **structure** for the blog, not the actual content. Focus on organizing the sections effectively and leaving room for creative content creation later.
    """

    # Use the wrapped LLM to generate the blog structure
    return wrapped_llm(blog_structure_prompt)
tools = [
    Tool(
        name="Main Index Retrieval",
        func=retrieve_relevant_chunks,
        args=["query", "pdf_file_path_1", "sentence-transformers/LaBSE"],  # specify required arguments
        description="Retrieve relevant chunks from the main pdf about insights on how to write blogs. The purpose is to assist in structuring the blog framework.",
    ),
    Tool(
        name="Side Index Retrieval",
        func=retrieve_relevant_chunks,
        args=["query", "pdf_file_path_2", "all-MiniLM-L6-v2"],  # specify required arguments
        description="Retrieve relevant chunks from the side pdf about insights on how to write blogs. This helps in building the blog layout.",
    ),
    Tool(
        name="LLM Query",
        func=query_llm,
        description="Query the LLM with context and prompt. This helps in guiding the agent's understanding of the query and structure.",
    ),
    Tool(
        name="Blog Structure Generator with Insights",
        func=generate_blog_structure_with_insights,
        description="Generate the structure for a blog based on the query for a trending topic using insights from PDFs. The focus is on laying out the blog’s sections, not writing content.",
    )
]


genai.configure(api_key=os.environ["GEMINI_API_KEY"])

class CustomGemini:
    """Custom class to interact with Google Gemini."""

    def __init__(self, temperature: float, max_tokens: int, model: str, google_api_key: str):
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.model = model
        genai.configure(api_key=google_api_key)
        self.generation_config = {
            "temperature": temperature,
            "max_output_tokens": max_tokens,
        }
        self.model_instance = genai.GenerativeModel(
            model_name=model,
            generation_config=self.generation_config,
        )

    def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = self.model_instance.generate_content([prompt])
        return response.text

custom_gemini = CustomGemini(
    temperature=0.7,
    max_tokens=512,
    model="gemini-1.5-flash",
    google_api_key=os.environ["GEMINI_API_KEY"]
)


class CustomLLMWrapper(LLM):
    custom_llm: CustomGemini

    @property
    def _llm_type(self) -> str:
        return "custom_gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self.custom_llm(prompt, stop=stop)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {
            "temperature": self.custom_llm.temperature,
            "max_tokens": self.custom_llm.max_tokens,
            "model": self.custom_llm.model,
        }

wrapped_llm = CustomLLMWrapper(custom_llm=custom_gemini)

agent = initialize_agent(
    tools=tools,
    llm=wrapped_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=5,
)


<ipython-input-22-bb1b43bddd65>:163: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [23]:
query = best_topic['query']
pdf_file_path_1 = "/content/CMI_Ultimate-Blogging-final.pdf"
pdf_file_path_2 = "/content/ImpulseCreative-BeginnersGuide-BusinessBlogging-2020.pdf"

response = agent.run(input=query, pdf_file_path_1=pdf_file_path_1, pdf_file_path_2=pdf_file_path_2)
print(response)


<ipython-input-23-c4748525a577>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(input=query, pdf_file_path_1=pdf_file_path_1, pdf_file_path_2=pdf_file_path_2)


The final answer is the blog structure provided by the Blog Structure Generator, as detailed in the Observation section above.  This structure includes an introduction defining "AI Humanizer," a body divided into sections focusing on AI as an assistant, AI mimicking human behavior, AI as a bridge, and AI augmenting human capabilities, and a conclusion summarizing the key points and offering a future outlook.  Each section includes suggested headlines and content ideas, emphasizing the use of data, examples, and narrative techniques to engage the reader.  The structure is designed to be informative and thought-provoking, exploring the multifaceted nature of AI's impact on humanity.


#**Generating a Blog**

In [29]:

class BlogState(BaseModel):
    topic: str
    structure: str  # Added structure field
    quotes: str = ""
    blog_content: str = ""
    final_blog: str = ""
    title: str = ""

# ✅ Define Gemini LLM
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

# ✅ Generate AI-Related Quotes
def generate_quotes(state: BlogState) -> BlogState:
    prompt = f"Provide 3-5 AI-related quotes relevant to the topic: {state.topic}"
    response = gemini_llm.invoke(prompt)
    state.quotes = response.content if isinstance(response, AIMessage) else str(response)
    return state

# ✅ Generate Blog Content using Topic & Structure
def write_blog(state: BlogState) -> BlogState:
    prompt = (
        f"Write a detailed, structured blog post about {state.topic} using the following structure:\n"
        f"{state.structure}\n\n"
        f"Use the following AI-related quotes to add credibility: {state.quotes}.\n"
        "Ensure it is engaging, informative, follows best SEO practices, and is between 1200-1800 words."
    )
    response = gemini_llm.invoke(prompt)
    state.blog_content = response.content if isinstance(response, AIMessage) else str(response)
    return state

# ✅ Enhance Readability & Formatting
def enhance_readability(state: BlogState) -> BlogState:
    prompt = (
        f"Improve readability & formatting of the following blog while making it engaging:\n"
        f"{state.blog_content}\n\n"
        "Ensure proper paragraph structuring, headings, bullet points, and optimize for SEO."
    )
    response = gemini_llm.invoke(prompt)
    state.final_blog = response.content if isinstance(response, AIMessage) else str(response)
    return state

def generate_title(state: BlogState) -> BlogState:
    prompt = (
        f"Create an eye-catching, SEO-optimized blog title for a post on {state.topic}, "
        "following this structure:\n"
        f"{state.structure}\n\n"
        "Ensure it's engaging and relevant to AI professionals."
    )
    response = gemini_llm.invoke(prompt)
    state.title = response.content if isinstance(response, AIMessage) else str(response)
    return state

# ✅ Define Workflow Graph
workflow = StateGraph(BlogState)
workflow.add_node("quote_agent", generate_quotes)
workflow.add_node("blog_writer", write_blog)
workflow.add_node("readability_agent", enhance_readability)
workflow.add_node("title_generator", generate_title)  # Added title generation step

# ✅ Define Execution Flow
workflow.set_entry_point("quote_agent")
workflow.add_edge("quote_agent", "blog_writer")
workflow.add_edge("blog_writer", "readability_agent")
workflow.add_edge("readability_agent", "title_generator")  # Title generation happens last

# ✅ Compile & Run Workflow
final_workflow = workflow.compile()

In [30]:
result = final_workflow.invoke(BlogState(topic=best_topic['query'], structure=response))

In [31]:
title= result["title"].replace("*", "")
content= result["final_blog"].replace("*", "")

#**Uploading The Blog**

In [32]:
from email import encoders
from email.mime.base import MIMEBase
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart



In [33]:

def send_email_with_attachment(sender_email, app_password, receiver_email, subject, body):
    try:
        # Create a multipart message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = subject

        # Attach the body with UTF-8 encoding
        msg.attach(MIMEText(body, 'plain', 'utf-8'))

        # Set up the SMTP server and send the email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Start TLS encryption
        server.login(sender_email, app_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())

        print(f"Email sent successfully to {receiver_email}!")
    except Exception as e:
        print(f"Error sending email: {e}")


In [34]:
send_email_with_attachment(YOUR_EMAIL,APP_PASSWORD,"EMAIL",title,content)

Email sent successfully to vexi413qefa@post.wordpress.com!
